## Import Libraries

In [1]:
import json
import uuid
import io
import copy
import re

In [2]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimlab.data_profile.' + cim_profile)
sparql = importlib.import_module('cimlab.loaders.sparql.' + cim_profile)

In [3]:
from cimlab.loaders import Parameter, ConnectionParameters
from cimlab.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimlab.models.model_parsers import add_to_catalog, add_to_typed_catalog, cim_dump, item_dump

from cimlab.models import NewModel

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

## Create New Feeder Model

In [5]:
reg = cim.GeographicalRegion(name = 'demo_region', mRID = str(uuid.uuid4()))
subreg = cim.SubGeographicalRegion(name = 'demo_subregion', mRID = str(uuid.uuid4()), Region=reg)
sub = cim.Substation(name = 'demo_sub', mRID = str(uuid.uuid4()), Region=subreg)
fdr = cim.Feeder(name = 'demo_feeder', mRID = str(uuid.uuid4()), NormalEnergizingSubstation=sub)

In [6]:
network = NewModel(bg)
network.Feeder = fdr
network.add_to_typed_catalog([reg, subreg, sub, fdr])

In [7]:
def create_line(network, name:str, bus1:str, bus2:str):
    line = cim.ACLineSegment(name = name, mRID = str(uuid.uuid4()), EquipmentContainer=network.Feeder)
    node1 = cim.ConnectivityNode(name = bus1, mRID = str(uuid.uuid4()), ConnectivityNodeContainer=network.Feeder)
    node2 = cim.ConnectivityNode(name = bus2, mRID = str(uuid.uuid4()), ConnectivityNodeContainer=network.Feeder)
    term1 = cim.Terminal(mRID = str(uuid.uuid4()))
    term2 = cim.Terminal(mRID = str(uuid.uuid4()))

    term1.ConnectivityNode = node1
    term2.ConnectivityNode = node2
    term1.ConductingEquipment = line
    term2.ConductingEquipment = line
    node1.Terminals.append(term1)
    node2.Terminals.append(term2)
    line.Terminals.append(term1)
    line.Terminals.append(term2)

    network.add_to_typed_catalog([line, node1, node2, term1, term2])

In [8]:
name = 'line_1'
bus1 = 'bus_1'
bus2 = 'bus_2'

create_line(network, name, bus1, bus2)

for line in network.typed_catalog[cim.ACLineSegment].values():
    line.r = 0.004
    line.x = 0.080
    line.b = 0.006

## Export and Upload

In [9]:
network.write_xml('demo_feeder.xml', "http://iec.ch/TC57/CIM100#")

In [10]:
network.upload()

/home/shared_user/.local/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


In [11]:
!python3 /home/shared_user/CIMHub/src_python/cimhub/ListFeeders.py

Feeder names and mRIDs:
ieee123 _C1C3E687-6FFD-C753-582B-632A27E28507
ieee123pv _E407CBB6-8C8D-9BC9-589C-AB83FBF0826D
ieee13nodeckt _49AD8E07-3BF9-A4E2-CB8F-C3722F837B62
ieee13nodecktassets _5B816B93-7A5F-B64C-8460-47C17D6E4B0F
ieee13ochre _13AD8E07-3BF9-A4E2-CB8F-C3722F837B62
ieee8500 _4F76A5F9-271D-9EB8-5E31-AA362D86F2C3
j1 _67AB291F-DCCD-31B7-B499-338206B9828F
ieee123transactive _503D6E20-F499-4CC7-8051-971E23D0BF79
final9500node _EE71F6C9-56F0-4167-A14E-7F4C71F10EAA
test9500new _AAE94E4A-2465-6F5E-37B1-3E72183A4E44
acep_psil _77966920-E1EC-EE8A-23EE-4EFD23B205BD
demo_feeder a10f57c0-00b4-4ef9-ad2e-dce88aa02f8c
maple10bus _9E985101-27AD-4FE4-B36A-EBECC98CDFAB
sourceckt _9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095


In [12]:
fdr.mRID

'a10f57c0-00b4-4ef9-ad2e-dce88aa02f8c'